In [44]:
import numpy as np
import math
import matplotlib.pyplot as plt 
from scipy import integrate
from math import log10, floor

def gen_latex_table(table,energies):
    print(r"\begin{table}[]" + "\n" + r"\resizebox{10.0cm}{!}{" + "\n" + r"\begin{tabular}{|c|    c|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}")
    print(r"\cline{3-15}")
    print(r"\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{13}{|c|}{Projectile Kinetic Energy (MeV)} \\")
    print(r"\cline{3-15}")
    header = "\multicolumn{1}{c}{}  & \multicolumn{1}{c}{}  & \multicolumn{1}{|c|}{4.0 MeV}"
    for i in energies:
        if (i==4.0):
            pass
        else:
            header += r" & ${0} \ MeV$".format(i)
    print(header + r" \\" )
    print(r"\cline{1-15}")
    print(r"\multirow{10}{*}{\rotatebox[origin=c]{90}{Projectile}}")
    for j in table:
        #j = j.replace("$1000.0^{\circ}$", "-")
        #j = j.replace("$-100.0\%$","-")
        #j = j.replace("$-1.0E+07$","-")
        #j = j.replace("E","x10^{")
        #j = j.replace("$-10000.0$", "-")
        print(j)
    print("\hline\n \end{tabular} \n } \n \end{table}")
    
def round_to_1(x):
    if x == 0:
        return 0
    else:
        return round(x, -int(floor(log10(abs(x)))))
    
def generate_dataDisplay(data,error):
    ''' Print data with error correctly'''
    error_str = str(error)
    integer_decimal = error_str.split(".")
    if error==0:
        return 0,0
    integer = integer_decimal[0]
    decimal = integer_decimal[1]
   # print( integer_decimal)
    if int(integer[0])>0:
        error = round(error,0)
        data = round(data,0)
        return int(data),int(error)
    else:
        for j in range(len(decimal)):
            if int(decimal[j]) != 0:
                error = round(error,(j+1))
                data = round(data,(j+1))
                return data,error
    
    
def data_formating(data,error,scientific_notation=False):
    if scientific_notation:
        n_error = len(str(int(data)))
        decimal_place = n_error - 1
        error_scientific = error/(10**(decimal_place))
        data_scientific = data/(10**(decimal_place))
        data_final, error_final = generate_dataDisplay(data_scientific, error_scientific)
        return r"({0} $\pm$ {1})E{2}".format(data_final,error_final,decimal_place)
    else:
        data_final, error_final = generate_dataDisplay(data,error)
        return r"{0} $\pm$ {1}".format(data_final,error_final)
        

def format_table(file,init,end):
    lines_data = []
    for i, line in enumerate(file):
        if i in np.arange(init, end): 
            lines_data.append(line) 
    data_array = np.loadtxt(lines_data)
    return data_array

def get_error(data,mean):
    summ = 0
    for i in data:
        summ += (i - mean)**2
    return math.sqrt(summ/(len(data) - 1))
    


def disp_vacancyCalc(path,n_files,ion,energy):
    """ Calculating the DPA throug the vacancy.txt """
    vacancy_ion_total = 0
    vacancyIon_array = []
    for i in range(n_files):
       
        file_path = path.replace("TRIM_outputs","TRIM_outputs_{0}".format(i+1))
        vacancy_file = open(file_path + "VACANCY.txt", 'r')
        repCol_file = open(file_path + "NOVAC.txt", 'r')
        vacancy_array = format_table(vacancy_file,init=34,end=134)
        repCol_array = format_table(repCol_file,init=29,end=129)
        #target_displacement = np.add(vacancy_array[:,5], repCol_array[:,1])
        target_displacement = vacancy_array[:,5]
        dpa_depth = dpa_depth_calc(target_displacement)
        #vacancy_plot = np.add(vacancy_array[:,1], vacancy_array[:,2])
        #vacancy_plot = np.add(vacancy_plot, vacancy_array[:,3])
        #vacancy_plot = np.add(vacancy_plot, vacancy_array[:,4])
        #vacancy_plot = np.add(vacancy_plot, vacancy_array[:,5])
        #plot(vacancy_array[:,0],vacancy_plot,file_path)
        #dpa = integral_calc(dpa_depth, vacancy_array[:,0])
        vacancy_ion = integral_calc(target_displacement, vacancy_array[:,0])
        vacancyIon_array.append(vacancy_ion)
        
        vacancy_ion_total += vacancy_ion
       
        
    vacancy_mean = vacancy_ion_total/n_files
    error = get_error(vacancyIon_array,vacancy_mean) 
    if (ion == "H" and energy == 30.0):
        flux_max = fluence_30mevProtonExp
        flux_error = 0
        percentage_error = 0
        return vacancy_mean,round_to_1(error),flux_max, percentage_error
    else:
        if vacancy_mean == 0:
            flux_max = 0
            flux_error = 0
            percentage_error = 0
        else:
            #flux_max = doping_concentration/vacancy_mean # ions/cm2
            flux_max = (vacancy_reference*fluence_reference)/(vacancy_mean)
            # get percentage of error from vacancy
            error = round_to_1(error)
            erro1 = (fluence_reference/vacancy_mean) # dx - 30mev proton
            erro2 = (vacancy_reference*fluence_reference)/(vacancy_mean**2) ##dy
            flux_error = math.sqrt( (erro1**2)*(vacancy_referenceError)**2 + (erro2**2)*(error)**2 ) 
            #percentage_error = (error/vacancy_mean)
            percentage_error = (flux_error/flux_max)
            print(percentage_error)
        
        return vacancy_mean,round_to_1(error),flux_max, percentage_error
           
def disp_depth_calc(T_dam,E_d=33):
    vacancy_ion = (0.8*(T_dam))/(2*E_d)
    '''
    i = 0
    T_dam = list(T_dam)
    while(i < len(T_dam)):
        T_dam[i] = (0.8*abs(T_dam[i]))/(2*E_d)
        i += 1
    '''
    return vacancy_ion

def dpa_depth_calc(y):
    i = 0
    y = list(y)
    while(i < len(y)):
        y[i] = (y[i]*1e8)/(si_atomic_density)
        i += 1
    return y
    
    
def integral_calc(y,x):
    # Get only data from 10um to 34 um.
    i = 0
    new_x = []
    new_y = []
    while(i < len(x)):
        if (x[i] >= 1e5) and (x[i] <= 3.4e5): # get the epitaxial layer region
            #print(yeap)
            new_x.append(x[i])
            new_y.append(y[i])
        i += 1
    #now integrate 
    dpa = integrate.simpson(new_y,new_x)
    return dpa    

def plot(x,y,name):
    plt.figure()
    plt.plot(x,y)
    plt.scatter(x,y)
    plt.xlabel("depth (Angstrom)")
    name = name.replace("TRIM_outputs_1/","")
    name = name.replace("/","")
    plt.ylabel("Vacancies (Number/Angstron.ion)")
    x_color = []
    y_color = [] 
    for i in range(len(x)):
        if (x[i] >= 1e5) and (x[i] <= 3.4e5):
            x_color.append(x[i])
            y_color.append(y[i])
    plt.fill_between(x_color, y_color, color='gray')
    plt.ylim(0)
   # plt.xlim(1e5,5e5)
    plt.savefig("plots/" + name + ".pdf")
    
ions = ["H", "He", "Li" ,"Be", "B", "C", "N", "O", "F", "Ne"]
#ions = ["H", "He", "Li"]
energies = [4.0,8.0,10.0,15.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0,100.0] # MeV
#ions = ["Be"]
#energies = [30.0] # MeV
nEvents=100000

table_dpaVac = []
table_dpaEn2 = []
table_fluxVac = []
table_flux_dpaEn2 = []

si_atomic_density = 4.977E22 #atoms/cm3
doping_concentration = 2.5E10 # n/cm2
fluence_30mevProtonExp = 1.5e13 # protons/cm2
file_path_reference = "TRIM_outputs/H_30.0MeV_n100000/"   
#get values of vacancy for 30 mev proton first because its the reference for all data
vacancy_reference, vacancy_referenceError, fluence_reference,fluence_referenceError =  disp_vacancyCalc(file_path_reference,3,"H",30.0)
print(vacancy_referenceError)

for ion in ions:
    line_dpaVac = "& " + ion
    line_dpaEn2 = "& " + ion
    line_fluxVac = "& " + ion
    line_flux_dpaEn2 = "& " + ion
    for energy in energies:
        if (ion=="H" or ion=="He" or ion=="Li"):
            nEvents=100000
        
        file_path = "TRIM_outputs/{0}_{1}MeV_n{2}/".format(ion,energy,nEvents)
        try:
            
            dpa_vacancy,error_vac,flux_max_vac,error_flux_vac = disp_vacancyCalc(file_path,3,ion,energy)
            #dpa_energy2,error_vacEn,flux_max_dpaEn2,error_flux_max_En = disp_energyMet2(file_path,3)
            #print(error_vac)
            vacancy_data = data_formating(dpa_vacancy,error_vac)
            #vacancy_en = data_formating(dpa_energy2,error_vacEn)

            #line_dpaVac += r" & {0:.2f} $\pm$ {1:.2f}".format(dpa_vacancy,error_vac)
            line_dpaVac += r" & {0}".format(vacancy_data)
            #line_fluxVac += r" & {0}".format(flux_vac_data)
            line_fluxVac += r" & {0:.1E} $\pm$ {1:.2f}\%".format(flux_max_vac, error_flux_vac*100)
            #line_dpaEn2 += " & {0}".format(vacancy_en)
            #line_flux_dpaEn2 += " & {0:.1E} $\pm$ {1:.2f}\%".format(flux_max_dpaEn2, error_flux_max_En*100)
            #print("{0} displacements/ion".format(dpa_vacancy))
        except Exception as ex:
            print(ex)
            pass
    line_dpaVac += r" \\"
   # line_dpaEn2 += r" \\"
    line_fluxVac += r" \\"
   # line_flux_dpaEn2 += r" \\"
    table_dpaVac.append(line_dpaVac)
   # table_dpaEn2.append(line_dpaEn2)
    table_fluxVac.append(line_fluxVac)
   # table_flux_dpaEn2.append(line_flux_dpaEn2)
    
print("------------------Tables-------------------")
print("------------------Vacancies/ion - VACANCY.txt-------------------") 
gen_latex_table(table_dpaVac,energies)
print("------------------Flux Max - VACANCY.txt-------------------")
gen_latex_table(table_fluxVac,energies)

'''
print("------------------DPA - ENERGY PHONONS ----------------------")
gen_latex_table(table_dpaEn2,energies)
print("------------------Flux Max - ENERGY PHONONS ----------------------")
gen_latex_table(table_flux_dpaEn2,energies)

print("------------------DPA - ENERGY METHOD 1-------------------")
gen_latex_table(table_dpaEn1,energies)
print("------------------DPA - ENERGY PHONONS ----------------------")
gen_latex_table(table_dpaEn2,energies)

#print("------------------DPA - ENERGY PHONONS ----------------------")
#gen_latex_table(table_dpaEn2,energies)
'''

'''
def disp_energyMet2(file_path,n_files): # REVIEW
    """ Calculating the DPA through the T_dam = E_phr - E_phi """
    vacancy_ion_total = 0
    vacancyIon_array = []
    file_name = file_path
    for i in range(n_files):
        filePath = file_path.replace("TRIM_outputs","TRIM_outputs_{0}".format(i+1))
        phonons_file = open(filePath + "PHONON.txt", 'r')
        phonons_array = format_table(phonons_file, init=27,end=137)
        E_phr = integral_calc(phonons_array[:,2],phonons_array[:,0])
        E_phi = integral_calc(phonons_array[:,1],phonons_array[:,0])
        T_dam = abs(E_phr - E_phi)
        vacancy_ion = disp_depth_calc(T_dam)
        vacancy_ion_total +=  vacancy_ion
        vacancyIon_array.append(vacancy_ion)
        
    vacancy_mean = vacancy_ion_total/n_files
    error = get_error(vacancyIon_array,vacancy_mean) 
    if vacancy_mean == 0:
        flux_max = 0
        flux_error = 0
        percentage_error = 0
    else:
        flux_max = doping_concentration/vacancy_mean # ions/cm2
        # get percentage of error from vacancy
        flux_error = (error/vacancy_mean)*flux_max
        percentage_error = (error/vacancy_mean)
    return vacancy_mean,round_to_1(error),flux_max, percentage_error
'''


print("End of Program")
        

0.009
0.0678534284677391
0.10492984878571955
0.08419531541417798
0.1067918358335815
0.10764384993424651
0.19770231705425287
0.1717951216819156
0.1479720042551646
0.19418317709318592
0.43533103815502994
0.2273588228933458
0.17222565402923928
0.06269251988899047
0.06624636177392991
0.06308067055205385
0.06418873547449669
0.06552161708048539
0.06459949582993053
0.0688632690651659
0.07597443022681157
0.06291215279079652
0.06312104598723818
0.0678058186667106
0.06568348188589336
0.06681416727144088
0.06274125075654957
0.06269028728744734
0.06305717119849734
0.06319358648692305
0.06307811232732247
0.06296472572968954
0.06774831383778956
0.06463504126746991
0.06528968713587051
0.07522528389197311
0.07863011742115045
0.06811005921718065
0.0626782072927867
0.06271574997172667
0.06269895898693291
0.06292679979736522
0.06315338865676248
0.06277126148739767
0.0628473999056105
0.0640045857372412
0.06283918445762664
0.06330662188576597
0.06582345539008357
0.063679889293196
0.0626830831381383
0.06269